In [2]:
import pandas as pd

Scrap the data frame from the Wikipedia page:

In [3]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]

Drop those rows where __Borough__ is <span style="color:blue">Not assigned</span>:

In [4]:
df = df[df['Borough'] != 'Not assigned']

If __Neighborhood__ is <span style="color:blue">Not assigned</span>, then set it to __Borough__:

In [5]:
df['Neighborhood'][df['Neighborhood'] == 'Not assigned'] = df['Borough']

Where there are multiple __Neighborhood__ entries for the same __Postcode__ and __Borough__, combine them in a single record:

In [6]:
df = df.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)).reset_index()

In [7]:
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


So what is the number of rows and columns in the dataframe?

In [8]:
print(df.shape)

(103, 3)


In [10]:
!conda install -c conda-forge geocoder --yes

Solving environment: \ ^C
failed

CondaError: KeyboardInterrupt



In [13]:
import geocoder as gc

ModuleNotFoundError: No module named 'geocoder'

In [24]:
postal_codes = pd.read_csv('http://cocl.us/Geospatial_data')

In [30]:
postal_codes.rename(columns={'Postal Code': 'Postcode'}, inplace=True)

In [31]:
postal_codes.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [35]:
new_df = pd.merge(df, postal_codes, how='right', on='Postcode')

In [36]:
new_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [37]:
new_df.shape

(103, 5)